In [1]:
import sys
import geopandas
import pandas as pd
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

## load GIS vector data

In [2]:
wo_geo = geopandas.read_file('/home/philipp/Data/edin_diss/GIS_wo/wo_2016_2020.gdb', layer='Stichtag_20160101')

In [3]:
wo_geo.head()

,FORSTBETRIEB_NR,REVIER_NR,ABTEILUNG,UNTERABTEILUNG,TEILFLAECHE,FARBCODE,LINKID,ID,VERWALTUNGSBLOCK,Erstellungsdatum,Datenaktualität,SHAPE_Length,SHAPE_Area,geometry
0,171,1,-1,0,1,00000000000,{7DDA8BDD-4E69-47D2-B7B5-D75FBCAAA0B2},1450126,529,2020-11-20T08:40:34,2016-01-01T00:00:00,195.920109,1553.41910,"MULTIPOLYGON (((616346.810 493670.595, 616342...."
1,171,1,-1,0,1,00000000000,{2CAE2042-5D26-4B61-857B-A3643351503C},1450044,529,2020-11-20T08:40:34,2016-01-01T00:00:00,1125.586021,35217.65235,"MULTIPOLYGON (((615325.040 492902.555, 615263...."
2,171,1,-1,0,1,00000000000,{62D0D4D2-FFD0-45DD-8622-5B24494B182C},1449991,529,2020-11-20T08:40:34,2016-01-01T00:00:00,946.503132,24184.69905,"MULTIPOLYGON (((608107.790 487993.105, 608108...."
3,171,1,-1,0,1,00000000000,{1AD6A0D3-C110-4E44-B7D8-DE5495E5C3EC},1450382,529,2020-11-20T08:40:34,2016-01-01T00:00:00,319.914046,5266.03840,"MULTIPOLYGON (((614628.190 492540.735, 614612...."
4,171,1,506,3,1,00000000003,{E77BB096-11C9-4939-9331-B4B305C4096A},1450402,529,2020-11-20T08:40:34,2016-01-01T00:00:00,2402.546849,11624.65550,"MULTIPOLYGON (((617139.640 493799.045, 617151...."


In [4]:
wo_geo.crs

{'init': 'epsg:31287'}

In [5]:
# filter abteilung -1
wo_geo = wo_geo.loc[wo_geo['ABTEILUNG'] != -1,:]

In [6]:
# create unique ID WO
wo_geo['WO'] = wo_geo['FORSTBETRIEB_NR'].astype(str) + \
wo_geo['REVIER_NR'].astype(str) + \
wo_geo['ABTEILUNG'].astype(str) + \
wo_geo['UNTERABTEILUNG'] + \
wo_geo['TEILFLAECHE'].astype(str)

## load SAP tax data

In [ ]:
# get infromation about which Teiloperate were active in a giveb year

In [19]:
# set year
year = 2016

path_sap_info = '/home/philipp/Data/edin_diss/SAP_tax/edin_meta_data.xlsx'
sap_info = pd.read_excel(path_sap_info)

In [8]:
sap_info.head()

,FB,FR,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,171,1,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
1,171,2,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
2,171,3,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
3,171,4,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356
4,171,5,1208,1208,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1030,1356,1356


In [16]:
# array with all TOs in a given year
tos = sap_info.loc[:,year].unique()

In [17]:
tos

array([1030, 1048, 1049, 1053, 1052, 1100, 1050, 1051, 1083, 1042, 1073,
       1092, 1074, 1075, 1151, 1194, 1086, 1043, 1144, 1197, 1055, 1057,
       1085, 1056, 1140, 1061, 1065, 1066, 1067, 1088, 1044, 1196, 1283,
       1031, 1041, 1079, 1093, 1045, 1081, 1082, 1069, 1070, 1071, 1156,
       1250])

In [26]:
sap_info.loc[sap_info[year] == 1030, 'FB'].unique()[0]

171

In [28]:
# set sap tax path directory
path_sap_tax_dir = '/home/philipp/Data/edin_diss/SAP_tax'

for to in tos:
    # get fb
    fb = sap_info.loc[sap_info[year] == to, 'FB'].unique()[0]
    # create path to file
    path_sap_tax_file = path_sap_tax_dir + '/' + str(fb) + '/TO_' + str(to) + '.XLS'
    
    print(path_sap_tax_file)


'''
wo_sap = pd.read_csv(path_sap_tax_file, 
                     sep='\t', 
                     encoding = "ISO-8859-1", 
                     decimal=',', 
                     error_bad_lines=False) '''

/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS
/home/philipp/Data/edin_diss/SAP_tax/171/TO_1048.XLS
/home/philipp/Data/edin_diss/SAP_tax/171/TO_1049.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1053.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1052.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1100.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1050.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1051.XLS
/home/philipp/Data/edin_diss/SAP_tax/172/TO_1083.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1042.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1073.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1092.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1074.XLS
/home/philipp/Data/edin_diss/SAP_tax/173/TO_1075.XLS
/home/philipp/Data/edin_diss/SAP_tax/174/TO_1151.XLS
/home/philipp/Data/edin_diss/SAP_tax/174/TO_1194.XLS
/home/philipp/Data/edin_diss/SAP_tax/175/TO_1086.XLS
/home/philipp/Data/edin_diss/SAP_tax/175/TO_1043.XLS
/home/philipp/Data/edin_diss/SAP_tax/175/TO_11

'\nwo_sap = pd.read_csv(path_sap_tax_file, \n                     sep=\'\t\', \n                     encoding = "ISO-8859-1", \n                     decimal=\',\', \n                     error_bad_lines=False) '

In [ ]:
'/home/philipp/Data/edin_diss/SAP_tax/171/TO_1030.XLS'